# Build Ye files

In this notebook, we demonstrate how to build Ye files with given prior netcdf files, given proxy meta data, and given PSMs.

In [1]:
%load_ext autoreload
%autoreload 2

from slmr import LMRt
from slmr import visual as vs

from pprint import pprint
import os
import numpy as np
import pandas as pd
import pickle
import time

import prysm

## Step 1: load proxies

In [2]:
job = LMRt.ReconJob()

pid=26242 >>> job.cfg created


In [3]:
print('Original settings:')
pprint(job.cfg.proxies.LMRdb.proxy_psm_type)

# update
job.cfg.proxies.LMRdb.proxy_psm_type['Tree Rings_WidthPages2'] = 'prysm.vslite'
job.cfg.proxies.LMRdb.proxy_psm_type['Ice Cores_d18O'] = 'prysm.ice.d18O'
job.cfg.proxies.LMRdb.proxy_psm_type['Corals and Sclerosponges_d18O'] = 'prysm.coral.d18O'

print('\nUpdated settings:')
pprint(job.cfg.proxies.LMRdb.proxy_psm_type)

Original settings:
{'Bivalve_d18O': 'linear',
 'Corals and Sclerosponges_Rates': 'linear',
 'Corals and Sclerosponges_SrCa': 'linear',
 'Corals and Sclerosponges_d18O': 'linear',
 'Ice Cores_Accumulation': 'linear',
 'Ice Cores_MeltFeature': 'linear',
 'Ice Cores_d18O': 'linear',
 'Ice Cores_dD': 'linear',
 'Lake Cores_BioMarkers': 'linear',
 'Lake Cores_GeoChem': 'linear',
 'Lake Cores_Misc': 'linear',
 'Lake Cores_Varve': 'linear',
 'Marine Cores_d18O': 'linear',
 'Speleothems_d18O': 'linear',
 'Tree Rings_Isotopes': 'linear',
 'Tree Rings_WidthBreit': 'bilinear',
 'Tree Rings_WidthPages2': 'bilinear',
 'Tree Rings_WoodDensity': 'linear'}

Updated settings:
{'Bivalve_d18O': 'linear',
 'Corals and Sclerosponges_Rates': 'linear',
 'Corals and Sclerosponges_SrCa': 'linear',
 'Corals and Sclerosponges_d18O': 'prysm.coral.d18O',
 'Ice Cores_Accumulation': 'linear',
 'Ice Cores_MeltFeature': 'linear',
 'Ice Cores_d18O': 'prysm.ice.d18O',
 'Ice Cores_dD': 'linear',
 'Lake Cores_BioMarkers':

In [4]:
# load proxies
dirpath = '/home/fzhu/SynologyDrive/Dev/Github/sLMR/slmr/LMRt/data'

db_proxies_filename = 'LMRdb_v0.4.0_Proxies.df.pckl'
db_metadata_filename = 'LMRdb_v0.4.0_Metadata.df.pckl'

job.load_proxies(
    os.path.join(dirpath, db_proxies_filename),
    os.path.join(dirpath, db_metadata_filename),
    seed=0, verbose=False, print_proxy_type_list=True,
)

pid=26242 >>> job.proxy_manager created

Proxy types
--------------
                                 Bivalve_d18O:    1
               Corals and Sclerosponges_Rates:    8
                Corals and Sclerosponges_SrCa:   25
                Corals and Sclerosponges_d18O:   60
                        Ice Cores_MeltFeature:    1
                               Ice Cores_d18O:   31
                                 Ice Cores_dD:    7
                              Lake Cores_Misc:    3
                             Lake Cores_Varve:    6
                       Tree Rings_WidthPages2:  347
                       Tree Rings_WoodDensity:   59
                                        TOTAL:  548


In [5]:
psm_keys = {                                                                                                                 
    'assim': list(set([pobj.psm_obj.psm_key for pobj in job.proxy_manager.sites_assim_proxy_objs])),                             
    'eval': list(set([pobj.psm_obj.psm_key for pobj in job.proxy_manager.sites_eval_proxy_objs])),                               
}     

print(psm_keys['assim'])

['linear', 'prysm.vslite', 'prysm.ice.d18O', 'prysm.coral.d18O']


## Step 2: build Ye files
1. Specify paths for prior files
2. run `job.build_ye_files()`, which has arguments as following:
    + ptype (str): the target proxy type                                                                                   
    + psm_name (str): the name of the PSM used to forward prior variables                                                  
    + prior_filesdict (dict): e.g. {'tas': tas_filepath, 'pr': pr_filepath}                                                
    + ye_savepath (str): the filepath to save precalculated Ye                                                             
    + psm_params (kwargs): the specific parameters for certain PSMs 

In [6]:
dirpath = '/home/fzhu/SynologyDrive/Dev/Github/LMRv2/input/data/model/icesm_last_millennium'

# atmosphere
tas_filename = 'tas_sfc_Amon_iCESM_past1000_085001-184912.nc'
pr_filename = 'pr_sfc_Amon_iCESM_past1000_085001-184912.nc'
psl_filename = 'psl_sfc_Amon_iCESM_past1000_085001-184912.nc'
d18O_filename = 'd18O_sfc_Amon_iCESM_past1000_085001-184912.nc'

# ocean
d18Ocoral_filename = 'd18Ocoral_sfc_Omon_iCESM_past1000_085001-184912.nc'
d18Osw_filename = 'd18Osw_sfc_Omon_iCESM_past1000_085001-184912.nc'
tos_filename = 'tos_sfc_Omon_iCESM_past1000_085001-184912.nc'  # sst
sos_filename = 'sos_sfc_Omon_iCESM_past1000_085001-184912.nc'  # sss

### Build Ye for tree-ring width with VS-Lite

In [7]:
%%time
prior_filesdict = {
    'tas': os.path.join(dirpath, tas_filename),
    'pr': os.path.join(dirpath, pr_filename),
}

ye_savepath =  '/home/fzhu/SynologyDrive/Dev/Github/sLMR/slmr/LMRt/data/Ye_prysm_tree_trw.npz'

job.build_ye_files(
    'Tree Rings_WidthPages2',
    'prysm.vslite',
    prior_filesdict, ye_savepath,
    Rlib_path='/home/fzhu/R/x86_64-pc-linux-gnu-library/3.4',
    vslite_params_path='/home/fzhu/SynologyDrive/Dev/Github/sLMR/slmr/LMRt/data/trw_calib/LMRdb_trw_params.pkl',
    verbose=True
)

Loading [tas] from /home/fzhu/SynologyDrive/Dev/Github/LMRv2/input/data/model/icesm_last_millennium/tas_sfc_Amon_iCESM_past1000_085001-184912.nc ...
Loading [pr] from /home/fzhu/SynologyDrive/Dev/Github/LMRv2/input/data/model/icesm_last_millennium/pr_sfc_Amon_iCESM_past1000_085001-184912.nc ...

Processing #1 - PAGES2kv2_NAm-nv512_NAm_049:trsgi ...
PRYSM >>> forward with prysm.vslite ...
PRYSM >>> Using R libs from: /home/fzhu/R/x86_64-pc-linux-gnu-library/3.4
PRYSM >>> T1=5.466, T2=10.828, M1=0.001, M2=0.275
PRYSM >>> Target: (40.2, 244.5); Found: (40.74, 245.00)
PRYSM >>> tas=257.9034423828125, pr=1.200515907839872e-05

Processing #2 - PAGES2kv2_Asia-QAMDJT_Asi_221:trsgi ...
PRYSM >>> forward with prysm.vslite ...
PRYSM >>> Using R libs from: /home/fzhu/R/x86_64-pc-linux-gnu-library/3.4
PRYSM >>> T1=6.922, T2=19.149, M1=0.046, M2=0.293
PRYSM >>> Target: (31.12, 97.03); Found: (31.26, 97.50)
PRYSM >>> tas=254.04881286621094, pr=8.74938541528536e-06

Processing #3 - PAGES2kv2_Asia-ONON

In [8]:
print(job.proxy_manager.all_proxies[0].psm_obj.psm_key)
print(job.proxy_manager.all_proxies[0].psm_obj.R)

prysm.vslite
None


### Build Ye for ice core d18O

In [18]:
%%time
prior_filesdict = {
    'tas': os.path.join(dirpath, tas_filename),
    'pr': os.path.join(dirpath, pr_filename),
    'psl': os.path.join(dirpath, psl_filename),
    'd18O': os.path.join(dirpath, d18O_filename),
}

ye_savepath =  '/home/fzhu/SynologyDrive/Dev/Github/sLMR/slmr/LMRt/data/Ye_prysm_ice_d18O.npz'

job.build_ye_files(
    'Ice Cores_d18O',
    'prysm.ice.d18O',
    prior_filesdict, ye_savepath,
     verbose=True
)

pid=12987 >>> Loading [tas] from /home/fzhu/SynologyDrive/Dev/Github/LMRv2/input/data/model/icesm_last_millennium/tas_sfc_Amon_iCESM_past1000_085001-184912.nc ...
pid=12987 >>> Loading [pr] from /home/fzhu/SynologyDrive/Dev/Github/LMRv2/input/data/model/icesm_last_millennium/pr_sfc_Amon_iCESM_past1000_085001-184912.nc ...
pid=12987 >>> Loading [psl] from /home/fzhu/SynologyDrive/Dev/Github/LMRv2/input/data/model/icesm_last_millennium/psl_sfc_Amon_iCESM_past1000_085001-184912.nc ...
pid=12987 >>> Loading [d18O] from /home/fzhu/SynologyDrive/Dev/Github/LMRv2/input/data/model/icesm_last_millennium/d18O_sfc_Amon_iCESM_past1000_085001-184912.nc ...

Processing #1 - PAGES2kv2_Ant-SiteDML07.Graf.2000_Ant_020:d18O ...
PRYSM >>> forward with prysm.ice.d18O ...
PRYSM >>> Target: (-75.58, 356.57); Found: (-74.84, 357.50)

Processing #2 - PAGES2kv2_Ant-PlateauRemot.Cole-Dai.1996_Ant_003:d18O ...
PRYSM >>> forward with prysm.ice.d18O ...
PRYSM >>> Target: (-84.0, 43.0); Found: (-84.32, 42.50)

Proc

## Build Ye for coral d18O

In [ ]:
%%time
prior_filesdict = {
    'sos': os.path.join(dirpath, sos_filename),
    'tos': os.path.join(dirpath, tos_filename),
    'd18Osw': os.path.join(dirpath, d18Osw_filename),
}

ye_savepath =  '/home/fzhu/SynologyDrive/Dev/Github/sLMR/slmr/LMRt/data/Ye_prysm_coral_d18O.npz'

job.build_ye_files(
    'Corals and Sclerosponges_d18O',
    'prysm.coral.d18O',
    prior_filesdict, ye_savepath,
    verbose=True
)

Loading [sos] from /home/fzhu/SynologyDrive/Dev/Github/LMRv2/input/data/model/icesm_last_millennium/sos_sfc_Omon_iCESM_past1000_085001-184912.nc ...


In [22]:
# load Ye
dirpath = '/home/fzhu/SynologyDrive/Dev/Github/sLMR/slmr/LMRt/data'
# ye_filename = '/home/fzhu/SynologyDrive/Dev/Github/sLMR/slmr/LMRt/data/Ye_prysm_tree_trw.npz'
ye_filename = '/home/fzhu/SynologyDrive/Dev/Github/sLMR/slmr/LMRt/data/Ye_prysm_tree_trw.npz'
# ye_filename = '/home/fzhu/SynologyDrive/Dev/Github/sLMR/slmr/LMRt/data/Ye_prysm_ice_d18O.npz'
linear_filename = 'icesm_last_millennium-tas_sfc_Amon-anom_linear_seasonPSM-GISTEMP_ref1951-1980_cal1850-2015_LMRdbv0.4.0.npz'
bilinear_filename = 'icesm_last_millennium-pr_sfc_Amon-tas_sfc_Amon-anom_bilinear_seasonPSM-T:GISTEMP-PR:GPCC_ref1951-1980_cal1850-2015_LMRdbv0.4.0.npz'

ye = np.load(os.path.join(dirpath, ye_filename))

# print(ye['pid_index_map'])
print(np.shape(ye['ye_vals']))
# print(ye['ye_vals'][0])

(347, 1000)
